# Combine Sub-SBML models with TXTL Machinery
---

Do this so you can incorporate temperature control <br>
2020.08.14

In [114]:
from biocrnpyler import *
#from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, Multimer
import numpy as np
#import pylab as plt 
#%matplotlib inline

import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
bokeh.io.output_notebook()

import colorcet

from subsbml import createNewSubsystem, createSubsystem

Loading BokehJS ...

In [115]:
palette = colorcet.b_glasbey_category10
palette.remove( '#d62628')

ValueError: list.remove(x): x not in list

## Combine atp synthase with txtl machinery 
---

In [116]:
# atp synthase models
ss1_atpsyn_txtl = createSubsystem('atpsynthase_machinery_txtl.xml', subsystemName = 'atpsyn_txtl')
ss2_atpsyn_mi = createSubsystem('atpsynthase_membrane_integration.xml', subsystemName = 'atpsyn_mi')
ss3_atpsynthesis = createSubsystem('atp_synthesis_through_atpsynthase.xml', subsystemName = 'atpsynthesis')
ss4_atp_use = createSubsystem('atp_use.xml', subsystemName = 'atp_use')

In [117]:
atp_subsystem_w_use = ss1_atpsyn_txtl + ss2_atpsyn_mi + ss3_atpsynthesis + ss4_atp_use

In [118]:
# SIMULATE WITH ATP SYNTHASE
# Initial conditions
spec_list_10 = ['dna_atpsyn', 'proton_outside_H', 'metabolite_atp',
                 'protein_RNAP_machinery', 
                'protein_Ribo_machinery', 'protein_RNAse_machinery']
for i,spec in enumerate(spec_list_10):
    atp_subsystem_w_use.setSpeciesAmount(spec,10)


# timepoints and get WITH ATP SYNTHASE
timepoints = np.linspace(0,31600,100)
results, _ = atp_subsystem_w_use.simulateWithBioscrape(timepoints, sbml_warnings = False)

# Get species IDs
ATP_id = atp_subsystem_w_use.getSpeciesByName('metabolite_atp').getId()
ADP_id = atp_subsystem_w_use.getSpeciesByName('metabolite_adp').getId()

outside_H_id = atp_subsystem_w_use.getSpeciesByName('proton_outside_H').getId()
inside_H_id = atp_subsystem_w_use.getSpeciesByName('proton_inside_H').getId()
atp_synthase_b_protein_id = atp_subsystem_w_use.getSpeciesByName('protein_atpsyn_b').getId()
atp_synthase_f_protein_id = atp_subsystem_w_use.getSpeciesByName('protein_atpsyn').getId()

odeint failed with mxstep=500...

In [119]:
# SIMULATE WITHOUT ATP SYNTHASE
#ss4_atp_use.setSpeciesAmount('proton_outside_H',10)
ss4_atp_use.setSpeciesAmount('metabolite_atp',10)

results2, _ = ss4_atp_use.simulateWithBioscrape(timepoints, sbml_warnings = False)

# GET ATP species ID
ATP_id2 = ss4_atp_use.getSpeciesByName('metabolite_atp').getId()

In [120]:
# PLOT WITH ATP SYNTHASE
p_w_atp = bokeh.plotting.figure(width = 450, height = 350, title = 'with ATP Synthase, ATP Use',
                          x_axis_label = 'time',
                          y_axis_label = 'concentration')
plot_list = [ATP_id, outside_H_id, inside_H_id, atp_synthase_b_protein_id]
plot_str = ['atp', 'outside_H', 'inside_H', 'bound atp synthase']
# plot_list = [atp_synthase_b_protein_id, atp_synthase_f_protein_id]
# plot_str = ['atp synthase bound', 'atp synthase free']
for i,item in enumerate(plot_list):
    p_w_atp.line(timepoints, results[item], line_width = 2, legend_label = plot_str[i], color = palette[i])
    

# PLOT WITHOUT ATP SYNTHASE
p_no_atp = bokeh.plotting.figure(width = 450, height = 350, title = 'only ATP Use',
                          x_axis_label = 'time',
                          y_axis_label = 'concentration')
p_no_atp.line(timepoints, results2[ATP_id2], line_width = 2, legend_label = 'atp', color = palette[0])

p_compare = bokeh.plotting.figure(width = 450, height = 350, title = 'Compare ATP',
                          x_axis_label = 'time',
                          y_axis_label = 'concentration')
p_compare.line(timepoints, results[ATP_id], legend_label = 'atp synthase atp', line_width = 2, color = palette[0])
p_compare.line(timepoints, results2[ATP_id2], legend_label = 'only atp', line_width = 2, color = palette[1])


p_w_atp.legend.location = 'bottom_right'
p_w_atp.legend.click_policy = 'hide'
    
bokeh.io.show(row(p_w_atp, p_no_atp))
bokeh.io.show(p_compare)




Why does bound atp synthase shoot infinitely high:( got em, it's because you kept remaking the transcript silly

## Combine all proton pump with txtl machinery
---

In [121]:
#ss5_pp_txtl = createSubsystem('proton_pump_tx_tl.xml', subsystemName = 'pp_txtl')
ss5_pp_txtl = createSubsystem('proton_pump_machinery_txtl.xml', subsystemName = 'pp_txtl')
ss6_pp_mi = createSubsystem('proton_pump_membrane_integration.xml', subsystemName = 'pp_mi')
ss7_protontransport = createSubsystem('proton_transport_through_proton_pump.xml', subsystemName = 'protontransport')

In [122]:
pp_subsystem = ss5_pp_txtl + ss6_pp_mi + ss7_protontransport

In [123]:
# Initial conditions
spec_list_10 = ['dna_proton_pump', 'proton_outside_H', 'metabolite_atp', 'protein_RNAP_machinery', 
                'protein_Ribo_machinery', 'protein_RNAse_machinery']
for i,spec in enumerate(spec_list_10):
    pp_subsystem.setSpeciesAmount(spec,10)


# timepoints and get results
timepoints = np.linspace(0,31600,100)
results, _ = pp_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

# Get species IDs
ATP_id = pp_subsystem.getSpeciesByName('metabolite_atp').getId()
ADP_id = pp_subsystem.getSpeciesByName('metabolite_adp').getId()

outside_H_id = pp_subsystem.getSpeciesByName('proton_outside_H').getId()
inside_H_id = pp_subsystem.getSpeciesByName('proton_inside_H').getId()
proton_pump_b_protein_id = pp_subsystem.getSpeciesByName('protein_proton_pump_b').getId()
proton_pump_f_protein_id = pp_subsystem.getSpeciesByName('protein_proton_pump').getId()



dna_pp = pp_subsystem.getSpeciesByName('dna_proton_pump').getId()

odeint failed with mxstep=500...

In [124]:
p_pp = bokeh.plotting.figure(width = 450, height = 350, title = 'with Proton Pump')
plot_list = [ATP_id, outside_H_id, inside_H_id, proton_pump_b_protein_id, dna_pp]
plot_str = ['atp', 'outside_H', 'inside_H', 'bound proton pump', 'dna protein pump']
for i,item in enumerate(plot_list):
    p_pp.line(timepoints, results[item], line_width = 2, legend_label = plot_str[i], color = palette[i])
    
p_pp.legend.click_policy = 'hide'
bokeh.io.show(p_pp)

## Combine atp synthase and proton pump subsystems (with machinery txtl)
---

In [125]:
entire_subsystem = ss1_atpsyn_txtl + ss2_atpsyn_mi + ss3_atpsynthesis + ss4_atp_use + ss5_pp_txtl + ss6_pp_mi + ss7_protontransport

In [134]:
# Initial conditions
spec_list_10 = ['dna_proton_pump', 'dna_atpsyn','proton_outside_H', 'metabolite_atp','protein_RNAP_machinery', 
                'protein_Ribo_machinery', 'protein_RNAse_machinery']
for i,spec in enumerate(spec_list_10):
    entire_subsystem.setSpeciesAmount(spec,10)


# timepoints and get results
# timepoints = np.linspace(0,10,100)
# results, _ = entire_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

# Get species IDs
ATP_id = entire_subsystem.getSpeciesByName('metabolite_atp').getId()
ADP_id = entire_subsystem.getSpeciesByName('metabolite_adp').getId()

outside_H_id = entire_subsystem.getSpeciesByName('proton_outside_H').getId()
inside_H_id = entire_subsystem.getSpeciesByName('proton_inside_H').getId()

proton_pump_b_protein_id = entire_subsystem.getSpeciesByName('protein_proton_pump_b').getId()
proton_pump_f_protein_id = entire_subsystem.getSpeciesByName('protein_proton_pump').getId()

atp_synthase_b_protein_id = entire_subsystem.getSpeciesByName('protein_atpsyn_b').getId()
atp_synthase_f_protein_id = entire_subsystem.getSpeciesByName('protein_atpsyn').getId()


In [135]:
timepoints = np.linspace(0,31600,100)
results_everything, _ = entire_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

odeint failed with mxstep=500...

In [136]:
p3 = bokeh.plotting.figure(width = 450, height = 350, title = 'with ATP Synthase, Proton Pump, ATP Use',
                          x_axis_label = 'time',
                          y_axis_label = 'concentration')
plot_list = [ATP_id, outside_H_id, inside_H_id, atp_synthase_b_protein_id, proton_pump_b_protein_id]
plot_str = ['atp', 'outside_H', 'inside_H', 'bound atp synthase', 'bound proton pump']
for i,item in enumerate(plot_list):
    p3.line(timepoints, results_everything[item], line_width = 2, legend_label = plot_str[i], color = palette[i])
    
p3.legend.click_policy = 'hide'
p3.legend.location = 'bottom_right'
bokeh.io.show(p3)

## Compare with atpsyn + proton pump, only atpsyn, only atpuse (with txtl machinery)
---

In [129]:
bokeh.io.show(row(p3,p_w_atp, p_no_atp))

Cool. What's going on with the second plot. How come it looks like atp isn't being consumed?? Fix those params?? Next do temperature sensitivity.

In [130]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,jupyterlab,scipy,biocrnpyler,bioscrape,colorcet,subsbml
  

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
bokeh 1.4.0
jupyterlab 1.2.6
scipy 1.4.1
biocrnpyler 0.2.1
bioscrape 1.0.0
colorcet 2.0.2
subsbml unknown
